# EEG Classification using CNN+LSTM model
Here we do hyperparameter grid search by making own GridSearch object and without using library functions or objects (such as GridSearchCV from sklearn). We need to create such an object, because it is not correct to compare neural networks by scores after a fixed number of epochs (due to overfiting and so on) and we need to plot learning curves.

In [1]:
import os
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import Callback, ProgbarLogger, BaseLogger
from keras import backend as K
from keras.regularizers import l1_l2

import data as dt

Using TensorFlow backend.


In [2]:
path_to_data = '/home/moskaleona/alenadir/data/rawData' #'C:/Users/alena/Desktop/homed/laba/data/rawData' 

In [3]:
data = dt.DataBuildClassifier(path_to_data).get_data([25, 33], shuffle=True, random_state=1, resample_to=128, windows=[(0.2, 0.5)],baseline_window=(0.2, 0.3))

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data[33][0], data[33][1], test_size=0.2, stratify=data[33][1], random_state=108)

In [5]:
from sklearn.metrics import roc_auc_score
import logging

class LossMetricHistory(Callback):
    def __init__(self, validation_data=(), verbose=1):
        super(LossMetricHistory, self).__init__()
        self.x_val, self.y_val = validation_data
        self.verbose = verbose
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.setLevel(logging.INFO)
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter("%(message)s")
        console.setFormatter(formatter)
        if len(self.logger.handlers) > 0:
            self.logger.handlers = []
        self.logger.addHandler(console)
            
    
    def on_train_begin(self, logs={}):
        self.logger.info("Training began")
        self.losses = []
        self.val_losses = []
        self.accs = []
        self.val_accs = []
        self.aucs = []
    
    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accs.append(logs.get('acc'))
        if self.x_val is not None and self.y_val is not None: 
            self.val_losses.append(logs.get('val_loss'))
            self.val_accs.append(logs.get('val_acc'))
            self.y_pred = self.model.predict_proba(self.x_val, verbose=0)
            self.aucs.append(roc_auc_score(self.y_val, self.y_pred))
            self.logger.info("epoch %d results: train loss = %.6f, val loss = %.6f"%(epoch + 1, self.losses[-1], self.val_losses[-1]) + 
                             "\n\t\t\tacc = %.6f, val acc = %.6f"%(self.accs[-1], self.val_accs[-1]) +
                             "\n\t\t\tauc = %.6f"%(self.aucs[-1]))
        else:
            self.logger.info("epoch %d results: train loss = %.6f"%(epoch + 1, self.losses[-1]) + 
                             "\n\t\t\tacc = %.6f"%(self.accs[-1]))
        

In [6]:
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
class CnnLstmClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, loss='binary_crossentropy', n_filters=10, n_lstm=30, n_iter=150, batch_size=10,
                 learning_rate=0.001, l1=0., l2=0.0, dropout=0., dropout_lstm=0., recurrent_dropout=0., threshold=0.5):
        self.loss = loss
        self.n_lstm = n_lstm
        self.n_filters = n_filters
        self.n_iter = n_iter
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.l1 = l1
        self.l2 = l2
        self.dropout = dropout
        self.dropout_lstm = dropout_lstm
        self.recurrent_dropout = recurrent_dropout
        self.threshold = threshold
    
    def _make_test_model(self, input_shape):
        self.model = Sequential()
        self.model.add(Dense(1, input_shape=(741,), activation='sigmoid'))
        
    def _make_model(self, input_shape, dropout, dropout_lstm, recurrent_dropout):
        batch_input_shape = (None, input_shape[1], input_shape[2])
        self.model = Sequential()
        self.model.add(Conv1D(self.n_filters, self.kernel_size_, batch_input_shape=batch_input_shape,
                         activation='relu', kernel_regularizer=l1_l2(self.l1, self.l2)))
        self.model.add(Dropout(dropout))
        self.model.add(LSTM(self.n_lstm,
                       dropout=dropout_lstm, recurrent_dropout=recurrent_dropout))
        self.model.add(Dense(1, activation='sigmoid'))
    
    def fit(self, X_train, y_train, X_val=None, y_val=None, verbose=1):
        # TODO: check the parameters
        self.kernel_size_ = X_train.shape[2]
        #self._make_test_model(X_train.shape)
        self._make_model(X_train.shape, self.dropout, self.dropout_lstm, self.recurrent_dropout)
        self.optimizer_ = RMSprop(lr=self.learning_rate)
        self.model.compile(loss=self.loss, optimizer=self.optimizer_, metrics=['acc'])

        #self.log_ = LossMetricHistory(validation_data=(X_val, y_val))#BaseLogger()
        self.hist_ = self.model.fit(X_train, y_train,
                        batch_size=self.batch_size,
                        epochs=self.n_iter) 
                        #validation_data=(X_val, y_val), verbose=verbose, callbacks=[self.log_])
        return self.hist_
    
    def predict(self, X):
        '''
        try:
            getattr(self, "kernel_size_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        '''
        proba = self.model.predict(X)
        return (proba > self.threshold).astype('int32')
    
    def predict_proba(self, X):
        '''
        try:
            getattr(self, "kernel_size_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        '''
        return self.model.predict(X)
    
    
    def score(self, X, y, scoring='auc'):
        try:
            if scoring=='auc':
                return roc_auc_score(y, self.predict_proba(X))
            elif scoring=='acc':
                return accuracy_score(y, self.predict(X))
            else:
                raise ValueError(message="No such option: '%s'. Use 'auc' or 'acc'"%str(scoring))
        except ValueError as err:
            print(err)
    

## Hyperparameter tuning

In [ ]:
class GridSearch:
    def __init__(self, model=None, ):